In [2]:
"""
Goal of this notebook is to take our x-ray png files and build a convolutional neural network to predict which images are part of 'findings' and which are part of 'no-findings'

Procedure and Outline

Loading the data
1. Use ImageDataGenerator to create augmentations of the data 
2. Split into training and validation subset

Training The Model
1. Load the pretrained models
2. Set trainable to False
3. Create a custom layer on top of this model -> Convolutional Layer, MaxPooling2D, Dense(128), Dense(1). 
4. * Experiment with dropout to see if it helps
5. Compile the model -> Set the learning rate (tf.keras.?.adam), loss='binary_crossentropy' since it is a binary classification problem, metric=['accuracy'] (or MatthewCorrelationCoefficient see step 1 of Testing)
6. Create history = model.fit(train=(X_train, y_train), validation=(X_val, y_val), epochs=5, verbose=1, batch_size=32)

Testing
1. Create a function to get the metric (using Matthew Correlation Coefficient) -> may have to do this prior to compiling the model and set this function as the metric. Look more into this, test both.
2. Create an array, y_test = model.predict(X_test)
3. Submit and see score

Assessment
Try to find weaknesses and see where you can Improve
"""

"\nGoal of this notebook is to take our x-ray png files and build a convolutional neural network to predict which images are part of 'findings' and which are part of 'no-findings'\n\nProcedure and Outline\n\nImage Dataset\n1. Load the image dataset\n2. Convert the dataset into a numpy array\n3. Define this array as X\n\nLabels\n1. Load the label dataset\n2. Take the column 'Finding' and convert that column alone into a numpy array. Since the column is already in order, there is an ordered bijective relationship between this column and the image dataset\n3. Define this column as Y\n\n Preprocessing\n1. Split X into X_train, X_val. X_train is first 8,000 images, and X_val is last 2,000\n2. Repeat for Y\n3. * Perform augmentation on the dataset, shift up/down/right/left, rotate, zoom in, change brightnes, flip on horizontal access. Do this step after you do first run through and see what difference it makes\n\nTraining The Model\n1. Load the pretrained res-net50 model\n2. Set trainable to

In [30]:
# import required libraries
import numpy as np
import pandas as pd
import os

# image processing
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# machine learning libraries
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications import DenseNet169
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input, BatchNormalization, Concatenate, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


In [2]:
train_file_path = '/kaggle/input/dataset/Train_PNG/Train_PNG'

datagen = ImageDataGenerator(
    rescale = 1.0/255.00,
    rotation_range = 10,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    zoom_range = 0.1,
    validation_split = 0.2
)

train_generator = datagen.flow_from_directory(
    directory = train_file_path,
    target_size = (512, 512),
    color_mode = 'rgb',
    class_mode = 'binary', 
    batch_size = 32,
    shuffle = False,
    subset = 'training'
)

validation_generator = datagen.flow_from_directory(
    directory = train_file_path,
    target_size = (512, 512),
    color_mode = 'rgb',
    class_mode = 'binary',
    batch_size = 32,
    shuffle = False,
    subset = 'validation'
)

Found 8001 images belonging to 2 classes.
Found 1999 images belonging to 2 classes.


In [32]:
input_layer = Input(shape=(512, 512, 3))

mobilenet_base = MobileNetV2(weights='imagenet', input_shape=(512, 512, 3), include_top=False)
densenet_base = DenseNet169(weights='imagenet', input_shape=(512, 512, 3), include_top=False)

# set the layers not trainable to preserve weights of pretrained model
for layer in mobilenet_base.layers:
    layer.trainable=False
for layer in densenet_base.layers:
    layer.trainable=False

model_mobilenet = mobilenet_base(input_layer) 
model_mobilenet = GlobalAveragePooling2D()(model_mobilenet)
output_mobilenet = Flatten()(model_mobilenet)

model_densenet = densenet_base(input_layer) 
model_densenet = GlobalAveragePooling2D()(model_densenet)
output_densenet = Flatten()(model_densenet)

merged = Concatenate()([output_mobilenet, output_densenet])

x = BatchNormalization()(merged)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(inputs=input_layer, outputs=x)

/tmp/ipykernel_36/1471134529.py:3: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  mobilenet_base = MobileNetV2(weights='imagenet', input_shape=(512, 512, 3), include_top=False)


In [33]:
for layer in base_model.layers[-10:]:
    layer.trainable = True

model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

early_stopping = EarlyStopping(patience=3, restore_best_weights=True)

lr_reducer = ReduceLROnPlateau(
    monitor='val_loss',
    patience=3,
    verbose=1,
    factor=0.5,
    min_lr=0.0001
)

checkpoint_callback = ModelCheckpoint(
    filepath='model_checkpoint.weights.h5',
    save_weights_only=True,
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)

In [34]:
history = model.fit(
    train_generator,
    epochs=5,
    batch_size=32,
    validation_data=validation_generator,
    verbose=1,
    callbacks=[checkpoint_callback, lr_reducer, early_stopping] # run again to see effects of lr_reducer and early_stopping
)

Epoch 1/5
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5665 - loss: 0.7983

2024-09-05 23:27:35.649208: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng3{k11=2} for conv (f32[15,128,128,128]{3,2,1,0}, u8[0]{0}) custom-call(f32[15,256,128,128]{3,2,1,0}, f32[128,256,1,1]{3,2,1,0}), window={size=1x1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convForward", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0}} is taking a while...
2024-09-05 23:27:35.769672: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.120573004s
Trying algorithm eng3{k11=2} for conv (f32[15,128,128,128]{3,2,1,0}, u8[0]{0}) custom-call(f32[15,256,128,128]{3,2,1,0}, f32[128,256,1,1]{3,2,1,0}), window={size=1x1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convForward", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"


Epoch 1: val_accuracy improved from -inf to 0.71236, saving model to model_checkpoint.weights.h5
251/251 ━━━━━━━━━━━━━━━━━━━━ 728s 3s/step - accuracy: 0.5667 - loss: 0.7980 - val_accuracy: 0.7124 - val_loss: 0.5800 - learning_rate: 1.0000e-05
Epoch 2/5
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6406 - loss: 0.6958
Epoch 2: val_accuracy improved from 0.71236 to 0.73037, saving model to model_checkpoint.weights.h5
251/251 ━━━━━━━━━━━━━━━━━━━━ 553s 2s/step - accuracy: 0.6408 - loss: 0.6955 - val_accuracy: 0.7304 - val_loss: 0.5191 - learning_rate: 1.0000e-05
Epoch 3/5
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6920 - loss: 0.6146
Epoch 3: val_accuracy improved from 0.73037 to 0.76488, saving model to model_checkpoint.weights.h5
251/251 ━━━━━━━━━━━━━━━━━━━━ 549s 2s/step - accuracy: 0.6921 - loss: 0.6144 - val_accuracy: 0.7649 - val_loss: 0.4822 - learning_rate: 1.0000e-05
Epoch 4/5
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7546 - loss: 0.5065
Epoch 4: val

In [38]:
test_file_path = '/kaggle/input/dataset/Test_PNG 2/Test_PNG'
test_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_generator = test_datagen.flow_from_directory(
    directory=test_file_path,
    target_size=(512, 512),
    color_mode='rgb',
    class_mode=None,
    batch_size=32,
    shuffle=False
)
predictions = model.predict(test_generator)

Found 2000 images belonging to 1 classes.
63/63 ━━━━━━━━━━━━━━━━━━━━ 63s 1s/step 


In [66]:
# could play around with this and see if it makes much difference in score
y_pred = np.where(predictions>0.32, 1, 0)

In [67]:
count0 = 0
count1 = 0
for i in range(len(y_pred)):
    if y_pred[i] == 0:
        count0 +=1
    else:
        count1 += 1
print(count0 / len(y_pred))
print(count1 / len(y_pred))

0.7075
0.2925


In [90]:
from datetime import datetime

y_submission = pd.read_csv('/kaggle/input/dataset/data (4).csv')
y_submission = y_submission[-2000:].reset_index(drop=True)
y_submission.loc[0:1999, 'Finding'] = y_pred
y_submission['id'] = y_submission['id'].apply(lambda x: str(x).zfill(5))
y_submission.rename(columns={'Finding': 'Outcome'}, inplace=True)
y_submission['Outcome'] = y_submission['Outcome'].astype(int)
y_submission

y_submission.to_csv(f"submit_{datetime.today().strftime('%Y-%m-%d_%H%M%S')}.csv", index=False)

In [82]:
import zipfile

y_submission.to_csv('submission.csv', index=False)
with zipfile.ZipFile('submission.zip', 'w') as zipf:
    zipf.write('submission.csv')